## 課題１

In [ ]:
%matplotlib inline
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os, glob, time, copy, random, zipfile
import matplotlib.pyplot as plt
from PIL import Image
from sklearn.model_selection import train_test_split
from tqdm import tqdm_notebook as tqdm


import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import torch.nn.functional as F
import torchvision
from torchvision import models, transforms

In [ ]:
torch.__version__

In [ ]:
# Check Current Directory
os.listdir('./dogs-vs-cats-reduced')


In [ ]:
# Train_dir, Test_dir
base_dir = './dogs-vs-cats-reduced'
train_dir = base_dir + '/train'
test_dir  = base_dir + '/test'

In [ ]:
# Check File Name
os.listdir(train_dir)[:5]

In [ ]:
# FilePath List
train_list = glob.glob(os.path.join(train_dir , '*.jpg'))
test_list = glob.glob(os.path.join(test_dir, '*.jpg'))

train_list = [string.replace('\\', '/') for string in train_list]
test_list = [string.replace('\\', '/') for string in test_list]

In [ ]:
img = Image.open(train_list[0])
plt.imshow(img)
plt.axis('off')
plt.show()

In [ ]:
img = Image.open(test_list[0])
plt.imshow(img)
plt.axis('off')
plt.show()

In [ ]:
# Label is contained in filepath
train_list[:5]

In [ ]:
# Image_Id is contained in filepath
test_list[:5]

In [ ]:
# Get Label
train_list[0].split('/')[-1].split('.')[0]

In [ ]:
# Get Image_Id
int(test_list[0].split('/')[-1].split('.')[0])

In [ ]:
# Number of Train Image
len(train_list)

In [ ]:
# Divide Train, Valid Data
train_list, val_list = train_test_split(train_list, test_size=0.1)

In [ ]:
print(len(train_list))
print(len(val_list))

In [ ]:
# Data Augumentation
class ImageTransform():
    
    def __init__(self, resize, mean, std):
        self.data_transform = {
            'train': transforms.Compose([
                transforms.RandomResizedCrop(resize, scale=(0.5, 1.0)),
                transforms.RandomHorizontalFlip(),
                transforms.ToTensor(),
                transforms.Normalize(mean, std)
            ]),
            'val': transforms.Compose([
                transforms.Resize(256),
                transforms.CenterCrop(resize),
                transforms.ToTensor(),
                transforms.Normalize(mean, std)
            ])
        }
        
    def __call__(self, img, phase):
        return self.data_transform[phase](img)

In [ ]:
# Dataset
class DogvsCatDataset(data.Dataset):
    
    def __init__(self, file_list, transform=None, phase='train'):    
        self.file_list = file_list
        self.transform = transform
        self.phase = phase
        
    def __len__(self):
        return len(self.file_list)
    
    def __getitem__(self, idx):
        
        img_path = self.file_list[idx]
        img = Image.open(img_path)
        
        img_transformed = self.transform(img, self.phase)
        
        # Get Label
        label = img_path.split('/')[-1].split('.')[0]
        if label == 'dog':
            label = 1
        elif label == 'cat':
            label = 0

        return img_transformed, label

In [ ]:
# Config
size = 224
mean = (0.485, 0.456, 0.406)
std = (0.229, 0.224, 0.225)
batch_size = 32
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
# Dataset
train_dataset = DogvsCatDataset(train_list, transform=ImageTransform(size, mean, std), phase='train')
val_dataset = DogvsCatDataset(val_list, transform=ImageTransform(size, mean, std), phase='val')

# Operation Check
print('Operation Check')
index = 0
print(train_dataset.__getitem__(index)[0].size())
print(train_dataset.__getitem__(index)[1])

In [ ]:
# DataLoader
train_dataloader = data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = data.DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

dataloader_dict = {'train': train_dataloader, 'val': val_dataloader}

# Operation Check
print('Operation Check')
batch_iterator = iter(train_dataloader)
inputs, label = next(batch_iterator)
print(inputs.size())
print(label)

In [ ]:
# VGG16 Model Loading
use_pretrained = True
net = models.vgg16(pretrained=use_pretrained)
print(net)

In [ ]:
# Change Last Layer
# Output Features 1000 → 2
net.classifier[6] = nn.Linear(in_features=4096, out_features=2)
print('Done')

In [ ]:
# Specify The Layers for updating
params_to_update = []

update_params_name = ['classifier.6.weight', 'classifier.6.bias']

for name, param in net.named_parameters():
    if name in update_params_name:
        param.requires_grad = True
        params_to_update.append(param)
        print(name)
    else:
        param.requires_grad = False

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(params=params_to_update, lr=0.001, momentum=0.9)

In [ ]:
def train_model(net, dataloader_dict, criterion, optimizer, num_epoch):
    
    since = time.time()
    best_model_wts = copy.deepcopy(net.state_dict())
    best_acc = 0.0
    net = net.to(device)
    
    for epoch in range(num_epoch):
        print('Epoch {}/{}'.format(epoch + 1, num_epoch))
        print('-'*20)
        
        for phase in ['train', 'val']:
            
            if phase == 'train':
                net.train()
            else:
                net.eval()
                
            epoch_loss = 0.0
            epoch_corrects = 0
            
            for inputs, labels in tqdm(dataloader_dict[phase]):
                inputs = inputs.to(device)
                labels = labels.to(device)
                optimizer.zero_grad()
                
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = net(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)
                    
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                        
                    epoch_loss += loss.item() * inputs.size(0)
                    epoch_corrects += torch.sum(preds == labels.data)
                    
            epoch_loss = epoch_loss / len(dataloader_dict[phase].dataset)
            epoch_acc = epoch_corrects.double() / len(dataloader_dict[phase].dataset)
            
            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))
            
            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(net.state_dict())
                
    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    net.load_state_dict(best_model_wts)
    return net

In [ ]:
# Train
num_epoch = 2
net = train_model(net, dataloader_dict, criterion, optimizer, num_epoch)

In [ ]:
# Prediction
id_list = []
pred_list = []

with torch.no_grad():
    for test_path in tqdm(test_list):
        img = Image.open(test_path)
        _id = int(test_path.split('/')[-1].split('.')[0])

        transform = ImageTransform(size, mean, std)
        img = transform(img, phase='val')
        img = img.unsqueeze(0)
        img = img.to(device)

        net.eval()

        outputs = net(img)
        preds = F.softmax(outputs, dim=1)[:, 1].tolist()
        
        id_list.append(_id)
        pred_list.append(preds[0])
    
    
res = pd.DataFrame({
    'id': id_list,
    'label': pred_list
})

res.sort_values(by='id', inplace=True)
res.reset_index(drop=True, inplace=True)

res.to_csv('submission_vgg16_reduced.csv', index=False)

In [ ]:
res.head(10)

In [ ]:
# Visualize Prediction
id_list = []
class_ = {0: 'cat', 1: 'dog'}

fig, axes = plt.subplots(2, 5, figsize=(20, 12), facecolor='w')

for ax in axes.ravel():
    
    i = random.choice(res['id'].values)
    
    label = res.loc[res['id'] == i, 'label'].values[0]
    if label > 0.5:
        label = 1
    else:
        label = 0
        
    img_path = os.path.join(test_dir, '{}.jpg'.format(i))
    img = Image.open(img_path)
    
    ax.set_title(class_[label])
    ax.imshow(img)

In [ ]:
# save model
torch.save(net.state_dict(), "./dogs-vs-cats-reduced/model_VGG16.pth")



## 課題２

In [ ]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn import ensemble
import numpy as np
from yellowbrick.datasets import load_occupancy
from yellowbrick.model_selection import FeatureImportances
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
sns.set()
dataset = datasets.load_iris()

In [ ]:
X = dataset.data
y = dataset.target
y_name = dataset.target_names

In [ ]:
# print("品種名: ", y_name)
# print("目的変数: ", y)
# print("説明変数: ", X)

In [ ]:
# データ２分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=0)

まずは普通にロジスティック回帰してみる

In [ ]:
# ロジスティック回帰
clf = linear_model.LogisticRegression(random_state=0)
clf.fit(X_train, y_train)

In [ ]:
# 学習結果
a = clf.coef_
b = clf.intercept_

# print("回帰係数: ", a)
# print("切片: ", b)

# その予測精度
print("正則化なしの訓練データでの点数: ", clf.score(X_train,y_train))
print("正則化なしのテストデータでの点数: ", clf.score(X_test, y_test))

L2正則化

In [ ]:
# ロジスティック回帰 + L2正則化
clf = linear_model.LogisticRegression(random_state=0, penalty='l2', n_jobs=1, C=1.0)
clf.fit(X_train, y_train)

In [ ]:
# 学習結果
a = clf.coef_
b = clf.intercept_

print("回帰係数: ", a)
print("切片: ", b)

# その予測精度
print("L2正則化の訓練データでの点数: ", clf.score(X_train,y_train))
print("L2正則のテストデータでの点数: ", clf.score(X_test, y_test))

L1正則化

In [ ]:
# ロジスティック回帰 + L1正則化
clf = linear_model.LogisticRegression(random_state=0, penalty='l1', n_jobs=1, C=1.0, solver='saga')
clf.fit(X_train, y_train)

In [ ]:
# 学習結果
a = clf.coef_
b = clf.intercept_

# print("回帰係数: ", a)
# print("切片: ", b)

# その予測精度
print("L2正則化の訓練データでの点数: ", clf.score(X_train,y_train))
print("L2正則のテストデータでの点数: ", clf.score(X_test, y_test))

ランダムフォレストで特徴量の重要度可視化

0=がく片の長さ，1=がく片の幅，2=花弁の長さ，3=花弁の幅 [cm]

In [ ]:
# 回帰木
rf = ensemble.RandomForestRegressor()

viz = FeatureImportances(rf)
viz.fit(X_train, y_train)
viz.show()

LASSO

In [ ]:
viz = FeatureImportances(clf, relative=False)
viz.fit(X_train, y_train)
viz.show()

## 課題３

In [ ]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn import ensemble
from yellowbrick.datasets import load_occupancy
from yellowbrick.model_selection import FeatureImportances

import os, sys
from collections import Counter
import numpy as np
import numpy.random as rd
import pandas as pd
import scipy as sp
from scipy import stats as st
pd.options.display.max_rows = 999

from sklearn.covariance import GraphicalLasso
from sklearn.preprocessing import StandardScaler
#from sklearn.covariance import GraphLasso

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from matplotlib import font_manager
import seaborn as sns
sns.set(style="whitegrid", palette="muted", color_codes=True)
%matplotlib inline

データ取得

In [ ]:
sns.set()
dataset = datasets.load_iris()

In [ ]:
X = dataset.data
label = ["sepal length", "sepal width", "petal length", "petal width"]
P = X.shape[1]
y = dataset.target
y_name = dataset.target_names

In [ ]:
img_path = "img/"
if not os.path.exists(img_path):
    os.mkdir(img_path)

# parametars
alpha = 0.2 # L１正則化パラメーター

# Scikit LearnのGraphical Lassoを使ってブロック降下法で分散共分散行列、精度行列を求める
model = GraphicalLasso(alpha=alpha,
                     max_iter=100,                     
                     verbose=True,
                     assume_centered = True)

model.fit(X)
cov_ = model.covariance_ # 分散共分散行列
prec_ = model.precision_ # 精度行列

# Scikit LearnのGraphical Lassoの結果表示
plt.figure(figsize=(10,4))
ax = plt.subplot(121)
sns.heatmap(cov_, annot=cov_, fmt='0.2f', ax=ax, xticklabels=label, yticklabels=label)
plt.title("Graphical Lasso: Covariance matrix")

ax = plt.subplot(122)
sns.heatmap(prec_, annot=prec_, fmt='0.2f', ax=ax, xticklabels=label, yticklabels=label)
plt.title("Graphical Lasso: Precision matrix")
plt.savefig(img_path+"glasso_cov_prec.png", dpi=128)
plt.show()

# 相関行列の算出
cor = np.empty_like(cov_)
for i in range(P):
    for j in range(P):
        cor[i, j] = cov_[i, j]/np.sqrt(cov_[i, i]*cov_[j, j])
        
# 偏相関行列の算出
rho = np.empty_like(prec_)
for i in range(P):
    for j in range(P):
        rho[i, j] = -prec_[i, j]/np.sqrt(prec_[i, i]*prec_[j, j])
        
plt.figure(figsize=(11,4))
ax = plt.subplot(122)
sns.heatmap(pd.DataFrame(rho), annot=rho, fmt='0.2f', ax=ax, xticklabels=label, yticklabels=label)
plt.title("Partial correlation Coefficiant with Scikit-Learn")
#plt.savefig(img_path+"partial_corr_sklearn.png", dpi=128)

ax = plt.subplot(121)
sns.heatmap(pd.DataFrame(cor), annot=cor, fmt='0.2f', ax=ax, xticklabels=label, yticklabels=label)
plt.title("Correlation Coefficiant with Scikit-Learn")
plt.savefig(img_path+"corr_pcorr_sklearn.png", dpi=128)
plt.show()

# ちゃんと単位行列になっているか確認してみる。
sns.heatmap(np.dot(cov_, prec_), annot=np.dot(cov_, prec_))
plt.savefig(img_path+"glasso_inv.png", dpi=128)

## 課題４

In [ ]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn import ensemble
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score  
from sklearn.metrics import recall_score   
from sklearn.metrics import f1_score  
from sklearn import metrics
from yellowbrick.datasets import load_occupancy
from yellowbrick.model_selection import FeatureImportances

import os
from contextlib import redirect_stdout

import sys
from collections import Counter
import numpy as np
import numpy.random as rd
import pandas as pd
import scipy as sp
from scipy import stats as st
pd.options.display.max_rows = 999

from sklearn.covariance import GraphicalLasso
from sklearn.preprocessing import StandardScaler
#from sklearn.covariance import GraphLasso

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from matplotlib import font_manager
import seaborn as sns
sns.set(style="whitegrid", palette="muted", color_codes=True)
%matplotlib inline

from sklearn.datasets import make_classification
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from collections import Counter
from imblearn.datasets import fetch_datasets
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import ClusterCentroids

In [ ]:
# 8.6:1
ecoli = fetch_datasets()['ecoli']
# ecoli.data.shape
X_ecoli = ecoli.data
y_ecoli = ecoli.target
print(sorted(Counter(y_ecoli).items()))

# 15:1
thyroid_sick = fetch_datasets()['thyroid_sick']
X_thyroid_sick = thyroid_sick.data
y_thyroid_sick = thyroid_sick.target
print(sorted(Counter(y_thyroid_sick).items()))

# 34:1
ozone_level = fetch_datasets()['ozone_level']
X_ozone_level = ozone_level.data
y_ozone_level = ozone_level.target
print(sorted(Counter(y_ozone_level).items()))

# 130:1
abalone_19 = fetch_datasets()['abalone_19']
X_abalone_19 = abalone_19.data
y_abalone_19 = abalone_19.target
print(sorted(Counter(y_abalone_19).items()))

下準備

以下の関数は、アルゴリズムの特性を説明するために、
リサンプリング後のサンプル空間をプロットするために使用されます。

In [ ]:
def plot_resampling(X, y, sampling, ax):
    X_res, y_res = sampling.fit_resample(X, y)
    ax.scatter(X_res[:, 0], X_res[:, 1], c=y_res, alpha=0.8, edgecolor='k')
    # make nice plotting
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.get_xaxis().tick_bottom()
    ax.get_yaxis().tick_left()
    ax.spines['left'].set_position(('outward', 10))
    ax.spines['bottom'].set_position(('outward', 10))
    return Counter(y_res)

In [ ]:
def plot_decision_function(X, y, clf, ax):
    plot_step = 0.02
    x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
    y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
    xx, yy = np.meshgrid(np.arange(x_min, x_max, plot_step),
                         np.arange(y_min, y_max, plot_step))

    Z = clf.predict(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)
    ax.contourf(xx, yy, Z, alpha=0.4)
    ax.scatter(X[:, 0], X[:, 1], alpha=0.8, c=y, edgecolor='k')

分類してみる

多次元だがx_0,x_1を取り出して２次元平面で表示

In [ ]:
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 12))

clf_ecoli = SVC(kernel='linear').fit(X_ecoli[:,:2], y_ecoli)
plot_decision_function(X_ecoli[:,:2],y_ecoli,clf_ecoli,ax1)
ax1.set_title('Linear SVC with y={}'.format(Counter(y_ecoli)))

clf_thyroid_sick: object = SVC(kernel='linear').fit(X_thyroid_sick[:,:2], y_thyroid_sick)
plot_decision_function(X_thyroid_sick[:,:2],y_thyroid_sick,clf_thyroid_sick,ax2)
ax2.set_title('Linear SVC with y={}'.format(Counter(y_thyroid_sick)))

clf_ozone_level = SVC(kernel='linear').fit(X_ozone_level[:,:2], y_ozone_level)
plot_decision_function(X_ozone_level[:,:2],y_ozone_level,clf_ozone_level,ax3)
ax3.set_title('Linear SVC with y={}'.format(Counter(y_ozone_level)))

clf_abalone_19 = SVC(kernel='linear').fit(X_abalone_19[:,:2], y_abalone_19)
plot_decision_function(X_abalone_19[:,:2],y_abalone_19,clf_abalone_19,ax4)
ax4.set_title('Linear SVC with y={}'.format(Counter(y_abalone_19)))

fig.tight_layout()

精度

In [ ]:
# 学習
clf_ecoli = SVC(kernel='linear',probability=True).fit(X_ecoli, y_ecoli)

In [ ]:
scores = {}

cmx_data = confusion_matrix(y_ecoli, clf_ecoli.predict(X_ecoli))
df_cmx = pd.DataFrame(cmx_data)
plt.figure(figsize=(3,3))
sns.heatmap(df_cmx, fmt='d', annot=True, square=True)
plt.title("ecoli")
plt.xlabel("predicted label")
plt.ylabel("true label")
plt.show()

metrics.plot_roc_curve(clf_ecoli, X_ecoli, y_ecoli)
plt.show()

scores[("ecoli",'accuracy')] = accuracy_score(y_ecoli, clf_ecoli.predict(X_ecoli))
scores[("ecoli","recall")] = recall_score(y_ecoli, clf_ecoli.predict(X_ecoli), pos_label=1)
scores[("ecoli","precision")] = precision_score(y_ecoli, clf_ecoli.predict(X_ecoli), pos_label=1)
scores[("ecoli","f1_score")] = f1_score(y_ecoli, clf_ecoli.predict(X_ecoli), pos_label=1)

In [ ]:
# 学習
clf_thyroid_sick = SVC(kernel='linear', probability=True).fit(X_thyroid_sick, y_thyroid_sick)

In [ ]:
cmx_data = confusion_matrix(y_thyroid_sick, clf_thyroid_sick.predict(X_thyroid_sick))
df_cmx = pd.DataFrame(cmx_data)
plt.figure(figsize=(3,3))
sns.heatmap(df_cmx, fmt='d', annot=True, square=True)
plt.title("thyroid")
plt.xlabel("predicted label")
plt.ylabel("true label")
plt.show()

metrics.plot_roc_curve(clf_thyroid_sick, X_thyroid_sick, y_thyroid_sick)
plt.show()

scores[("thyroid",'accuracy')] = accuracy_score(y_thyroid_sick, clf_thyroid_sick.predict(X_thyroid_sick))
scores[("thyroid","recall")] = recall_score(y_thyroid_sick, clf_thyroid_sick.predict(X_thyroid_sick), pos_label=1)
scores[("thyroid","precision")] = precision_score(y_thyroid_sick, clf_thyroid_sick.predict(X_thyroid_sick), pos_label=1)
scores[("thyroid","f1_score")] = f1_score(y_thyroid_sick, clf_thyroid_sick.predict(X_thyroid_sick), pos_label=1)

In [ ]:
# 学習
clf_ozone_level = SVC(kernel='linear',probability=True).fit(X_ozone_level, y_ozone_level)

In [ ]:
cmx_data = confusion_matrix(y_ozone_level, clf_ozone_level.predict(X_ozone_level))
df_cmx = pd.DataFrame(cmx_data)
plt.figure(figsize=(3,3))
sns.heatmap(df_cmx, fmt='d', annot=True, square=True)
plt.title("ozone_level")
plt.xlabel("predicted label")
plt.ylabel("true label")
plt.show()

metrics.plot_roc_curve(clf_ozone_level, X_ozone_level, y_ozone_level)
plt.show()

scores[("ozone_level",'accuracy')] = accuracy_score(y_ozone_level, clf_ozone_level.predict(X_ozone_level))
scores[("ozone_level","recall")] = recall_score(y_ozone_level, clf_ozone_level.predict(X_ozone_level), pos_label=1)
scores[("ozone_level","precision")] = precision_score(y_ozone_level, clf_ozone_level.predict(X_ozone_level), pos_label=1)
scores[("ozone_level","f1_score")] = f1_score(y_ozone_level, clf_ozone_level.predict(X_ozone_level), pos_label=1)

In [ ]:
# 学習
clf_abalone_19 = SVC(kernel='linear',probability=True).fit(X_abalone_19, y_abalone_19)

In [ ]:
cmx_data = confusion_matrix(y_abalone_19, clf_abalone_19.predict(X_abalone_19))
df_cmx = pd.DataFrame(cmx_data)
plt.figure(figsize=(3,3))
sns.heatmap(df_cmx, fmt='d', annot=True, square=True)
plt.title("abalone_19")
plt.xlabel("predicted label")
plt.ylabel("true label")
plt.show()

metrics.plot_roc_curve(clf_abalone_19, X_abalone_19, y_abalone_19)
plt.show()

scores[("abalone_19",'accuracy')] = accuracy_score(y_abalone_19, clf_abalone_19.predict(X_abalone_19))
scores[("abalone_19","recall")] = recall_score(y_abalone_19, clf_abalone_19.predict(X_abalone_19), pos_label=1)
scores[("abalone_19","precision")] = precision_score(y_abalone_19, clf_abalone_19.predict(X_abalone_19), pos_label=1)
scores[("abalone_19","f1_score")] = f1_score(y_abalone_19, clf_abalone_19.predict(X_abalone_19), pos_label=1)

精度(Accuracy)：真陽性と真陰性の割合の大きさ

適合率(Precision)：正と予測したもののうち実際に正だった割合

再現率(Recall)：正だったもののうち正と予測されていたものの割合

F値：再現率と適合率の調和平均

In [ ]:
print("Metrics:")
print(pd.Series(scores).unstack())

オーバーサンプリング：トレーニングデータの少ないクラスを疑似的に増やす

In [ ]:
scores_oversampled = {}

ros = RandomOverSampler(random_state=0)
X_resampled, y_resampled = ros.fit_resample(X_ecoli,y_ecoli)

clf_resampled = SVC(kernel='linear',probability=True).fit(X_resampled, y_resampled)

In [ ]:
cmx_data = confusion_matrix(y_resampled, clf_resampled.predict(X_resampled))
df_cmx = pd.DataFrame(cmx_data)
plt.figure(figsize=(3,3))
sns.heatmap(df_cmx, fmt='d', annot=True, square=True)
plt.title("ecoli oversampled")
plt.xlabel("predicted label")
plt.ylabel("true label")
plt.show()

metrics.plot_roc_curve(clf_resampled, X_resampled, y_resampled)
plt.show()

scores_oversampled[("ecoli",'accuracy')] = accuracy_score(y_resampled, clf_resampled.predict(X_resampled))
scores_oversampled[("ecoli","recall")] = recall_score(y_resampled, clf_resampled.predict(X_resampled), pos_label=1)
scores_oversampled[("ecoli","precision")] = precision_score(y_resampled, clf_resampled.predict(X_resampled), pos_label=1)
scores_oversampled[("ecoli","f1_score")] = f1_score(y_resampled, clf_resampled.predict(X_resampled), pos_label=1)

In [ ]:
fig, ax1= plt.subplots(1, 1, figsize=(3, 3))

clf_resampled = SVC(kernel='linear').fit(X_resampled[:,:2], y_resampled)
plot_decision_function(X_resampled[:,:2],y_resampled,clf_resampled,ax1)
ax1.set_title('Linear SVC with y={}'.format(Counter(y_resampled)))

In [ ]:
X_resampled, y_resampled = ros.fit_resample(X_thyroid_sick,y_thyroid_sick)

clf_resampled = SVC(kernel='linear',probability=True).fit(X_resampled, y_resampled)

In [ ]:
cmx_data = confusion_matrix(y_resampled, clf_resampled.predict(X_resampled))
df_cmx = pd.DataFrame(cmx_data)
plt.figure(figsize=(3,3))
sns.heatmap(df_cmx, fmt='d', annot=True, square=True)
plt.title("thyroid_sick oversampled")
plt.xlabel("predicted label")
plt.ylabel("true label")
plt.show()

metrics.plot_roc_curve(clf_resampled, X_resampled, y_resampled)
plt.show()

scores_oversampled[("thyroid_sick",'accuracy')] = accuracy_score(y_resampled, clf_resampled.predict(X_resampled))
scores_oversampled[("thyroid_sick","recall")] = recall_score(y_resampled, clf_resampled.predict(X_resampled), pos_label=1)
scores_oversampled[("thyroid_sick","precision")] = precision_score(y_resampled, clf_resampled.predict(X_resampled), pos_label=1)
scores_oversampled[("thyroid_sick","f1_score")] = f1_score(y_resampled, clf_resampled.predict(X_resampled), pos_label=1)

In [ ]:
X_resampled, y_resampled = ros.fit_resample(X_ozone_level,y_ozone_level)

clf_resampled = SVC(kernel='linear',probability=True).fit(X_resampled, y_resampled)

In [ ]:
cmx_data = confusion_matrix(y_resampled, clf_resampled.predict(X_resampled))
df_cmx = pd.DataFrame(cmx_data)
plt.figure(figsize=(3,3))
sns.heatmap(df_cmx, fmt='d', annot=True, square=True)
plt.title("ozone_level oversampled")
plt.xlabel("predicted label")
plt.ylabel("true label")
plt.show()

metrics.plot_roc_curve(clf_resampled, X_resampled, y_resampled)
plt.show()

scores_oversampled[("ozone_level",'accuracy')] = accuracy_score(y_resampled, clf_resampled.predict(X_resampled))
scores_oversampled[("ozone_level","recall")] = recall_score(y_resampled, clf_resampled.predict(X_resampled), pos_label=1)
scores_oversampled[("ozone_level","precision")] = precision_score(y_resampled, clf_resampled.predict(X_resampled), pos_label=1)
scores_oversampled[("ozone_level","f1_score")] = f1_score(y_resampled, clf_resampled.predict(X_resampled), pos_label=1)

In [ ]:
X_resampled, y_resampled = ros.fit_resample(X_abalone_19,y_abalone_19)

clf_resampled = SVC(kernel='linear',probability=True).fit(X_resampled, y_resampled)

In [ ]:
cmx_data = confusion_matrix(y_resampled, clf_resampled.predict(X_resampled))
df_cmx = pd.DataFrame(cmx_data)
plt.figure(figsize=(3,3))
sns.heatmap(df_cmx, fmt='d', annot=True, square=True)
plt.title("abalone_19 oversampled")
plt.xlabel("predicted label")
plt.ylabel("true label")
plt.show()

metrics.plot_roc_curve(clf_resampled, X_resampled, y_resampled)
plt.show()

scores_oversampled[("abalone_19",'accuracy')] = accuracy_score(y_resampled, clf_resampled.predict(X_resampled))
scores_oversampled[("abalone_19","recall")] = recall_score(y_resampled, clf_resampled.predict(X_resampled), pos_label=1)
scores_oversampled[("abalone_19","precision")] = precision_score(y_resampled, clf_resampled.predict(X_resampled), pos_label=1)
scores_oversampled[("abalone_19","f1_score")] = f1_score(y_resampled, clf_resampled.predict(X_resampled), pos_label=1)

以下にオーバーサンプル前後の指標を比較する

精度は落ちるものの他の指標が非常に改善されている．

In [ ]:
print("Previous Metrics:")
print(pd.Series(scores).unstack())

print("Oversampled Metrics")
print(pd.Series(scores_oversampled).unstack())

アンダーサンプリング：データの多い方を擬似的に減らす

In [ ]:
scores_undersampled = {}

cc = ClusterCentroids(random_state=0)
X_resampled, y_resampled = cc.fit_resample(X_ecoli,y_ecoli)

clf_resampled = SVC(kernel='linear',probability=True).fit(X_resampled, y_resampled)

In [ ]:
cmx_data = confusion_matrix(y_resampled, clf_resampled.predict(X_resampled))
df_cmx = pd.DataFrame(cmx_data)
plt.figure(figsize=(3,3))
sns.heatmap(df_cmx, fmt='d', annot=True, square=True)
plt.title("ecoli undersampled")
plt.xlabel("predicted label")
plt.ylabel("true label")
plt.show()

metrics.plot_roc_curve(clf_resampled, X_resampled, y_resampled)
plt.show()

scores_undersampled[("ecoli",'accuracy')] = accuracy_score(y_resampled, clf_resampled.predict(X_resampled))
scores_undersampled[("ecoli","recall")] = recall_score(y_resampled, clf_resampled.predict(X_resampled), pos_label=1)
scores_undersampled[("ecoli","precision")] = precision_score(y_resampled, clf_resampled.predict(X_resampled), pos_label=1)
scores_undersampled[("ecoli","f1_score")] = f1_score(y_resampled, clf_resampled.predict(X_resampled), pos_label=1)

In [ ]:
X_resampled, y_resampled = cc.fit_resample(X_thyroid_sick,y_thyroid_sick)

clf_resampled = SVC(kernel='linear',probability=True).fit(X_resampled, y_resampled)

In [ ]:
cmx_data = confusion_matrix(y_resampled, clf_resampled.predict(X_resampled))
df_cmx = pd.DataFrame(cmx_data)
plt.figure(figsize=(3,3))
sns.heatmap(df_cmx, fmt='d', annot=True, square=True)
plt.title("thyroid_sick undersampled")
plt.xlabel("predicted label")
plt.ylabel("true label")
plt.show()

metrics.plot_roc_curve(clf_resampled, X_resampled, y_resampled)
plt.show()

scores_undersampled[("thyroid_sick",'accuracy')] = accuracy_score(y_resampled, clf_resampled.predict(X_resampled))
scores_undersampled[("thyroid_sick","recall")] = recall_score(y_resampled, clf_resampled.predict(X_resampled), pos_label=1)
scores_undersampled[("thyroid_sick","precision")] = precision_score(y_resampled, clf_resampled.predict(X_resampled), pos_label=1)
scores_undersampled[("thyroid_sick","f1_score")] = f1_score(y_resampled, clf_resampled.predict(X_resampled), pos_label=1)

In [ ]:
X_resampled, y_resampled = cc.fit_resample(X_ozone_level,y_ozone_level)

clf_resampled = SVC(kernel='linear',probability=True).fit(X_resampled, y_resampled)

In [ ]:
cmx_data = confusion_matrix(y_resampled, clf_resampled.predict(X_resampled))
df_cmx = pd.DataFrame(cmx_data)
plt.figure(figsize=(3,3))
sns.heatmap(df_cmx, fmt='d', annot=True, square=True)
plt.title("ozone_level undersampled")
plt.xlabel("predicted label")
plt.ylabel("true label")
plt.show()

metrics.plot_roc_curve(clf_resampled, X_resampled, y_resampled)
plt.show()

scores_undersampled[("ozone_level",'accuracy')] = accuracy_score(y_resampled, clf_resampled.predict(X_resampled))
scores_undersampled[("ozone_level","recall")] = recall_score(y_resampled, clf_resampled.predict(X_resampled), pos_label=1)
scores_undersampled[("ozone_level","precision")] = precision_score(y_resampled, clf_resampled.predict(X_resampled), pos_label=1)
scores_undersampled[("ozone_level","f1_score")] = f1_score(y_resampled, clf_resampled.predict(X_resampled), pos_label=1)


In [ ]:
X_resampled, y_resampled = cc.fit_resample(X_abalone_19,y_abalone_19)

clf_resampled = SVC(kernel='linear',probability=True).fit(X_resampled, y_resampled)

In [ ]:
cmx_data = confusion_matrix(y_resampled, clf_resampled.predict(X_resampled))
df_cmx = pd.DataFrame(cmx_data)
plt.figure(figsize=(3,3))
sns.heatmap(df_cmx, fmt='d', annot=True, square=True)
plt.title("abalone_19 undersampled")
plt.xlabel("predicted label")
plt.ylabel("true label")
plt.show()

metrics.plot_roc_curve(clf_resampled, X_resampled, y_resampled)
plt.show()

scores_undersampled[("abalone_19",'accuracy')] = accuracy_score(y_resampled, clf_resampled.predict(X_resampled))
scores_undersampled[("abalone_19","recall")] = recall_score(y_resampled, clf_resampled.predict(X_resampled), pos_label=1)
scores_undersampled[("abalone_19","precision")] = precision_score(y_resampled, clf_resampled.predict(X_resampled), pos_label=1)
scores_undersampled[("abalone_19","f1_score")] = f1_score(y_resampled, clf_resampled.predict(X_resampled), pos_label=1)

以下にサンプル加工前後の指標を比較する

In [ ]:
print("Previous Metrics:")
print(pd.Series(scores).unstack())

print("Oversampled Metrics")
print(pd.Series(scores_oversampled).unstack())

print("Undersampled Metrics")
print(pd.Series(scores_undersampled).unstack())

指標を見ると，ecoli や abalone_19 データセットではオーバーサンプリングが，
ozone_level や thyroid_sick データセットではアンダーサンプリングが良い改善をしている．

## 課題５

## 課題５－１：特徴選択

まず yellowbrick の 再帰特徴削減(RFE) を走らせてみる．

再帰的特徴除去（RFE）は、モデルを適合させ、
指定された特徴数に達するまで最も弱い特徴（または特徴）を除去する特徴選択法です。
特徴はモデルのcoef_またはfeature_importances_属性によってランク付けされ、
ループごとに少数の特徴を再帰的に除去することで、
RFEはモデルに存在する可能性のある依存性と共線性を除去しようとします。

RFEでは、保持するために指定された数の特徴を必要としますが、
どれだけの特徴が有効であるかは事前にわからないことが多いです。
最適な特徴の数を見つけるために、RFEでクロスバリデーションを使用して、
異なる特徴のサブセットをスコア化し、最もスコア化された特徴の集合を選択します。
RFECV ビジュアライザーは、モデル内の特徴の数をクロスバリデーションされたテスト・スコアと
ばらつきとともにプロットし、選択された特徴の数を可視化します。

これが実際にどのように機能するかを示すために、
25個のうち情報量の多い特徴を3個しか持たないデータセットを使った例から始めます。

In [ ]:
from sklearn.svm import SVC
from sklearn.datasets import make_classification

from yellowbrick.model_selection import RFECV

# Create a dataset with only 3 informative features
X, y = make_classification(
    n_samples=1000, n_features=25, n_informative=3, n_redundant=2,
    n_repeated=0, n_classes=8, n_clusters_per_class=1, random_state=0
)

# Instantiate RFECV visualizer with a linear SVM classifier
visualizer = RFECV(SVC(kernel='linear', C=1))

visualizer.fit(X, y)        # Fit the data to the visualizer
visualizer.show()           # Finalize and render the figure

この図は理想的なRFECV曲線を示しており、3つの情報的特徴が捕捉されると曲線は優れた精度にジャンプし、
その後、非情報的特徴がモデルに追加されると徐々に精度が低下します。
網掛けの領域は、クロスバリデーションの変動性を表し、
曲線によって描かれた平均精度スコアの上と下の1標準偏差を表しています。

実際のデータセットを探索すると、信用デフォルトのバイナリ分類器でのRFECVの影響を見ることができます。

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold

from yellowbrick.model_selection import RFECV
from yellowbrick.datasets import load_credit

# Load classification dataset
X, y = load_credit()

cv = StratifiedKFold(5)
visualizer = RFECV(RandomForestClassifier(), cv=cv, scoring='f1_weighted')

visualizer.fit(X, y)        # Fit the data to the visualizer
visualizer.show()           # Finalize and render the figure

この例では、19個の特徴が選択されていることがわかりますが、
5個程度の特徴の後ではモデルのf1スコアはあまり改善されていないように見えます。
除去する特徴の選択は、各再帰の結果を決定する上で大きな役割を果たします。
各ステップで複数の特徴を除去するようにステップパラメータを変更すると、
最悪の特徴を早期に除去し、残りの特徴を強化するのに役立つかもしれません
（また、特徴の数が多いデータセットの特徴除去を高速化するために使用することもできます）。

### UCIのデータセットなど選んで再帰特徴削減(RFE) をやってみる

yellowbrick のデータセットにある occupancy データセットを取り出して実行

In [ ]:
from yellowbrick.datasets import load_occupancy

X, y = load_occupancy()

cv = StratifiedKFold(5)
visualizer = RFECV(RandomForestClassifier(), cv=cv, scoring='f1_weighted')

visualizer.fit(X, y)        # Fit the data to the visualizer
visualizer.show()    

### 課題５－２：検証曲線・学習曲線

まず yellowbrick の ’learning curve’ での例を走らせる．

学習曲線は、訓練サンプル数が変化する推定量について、訓練スコアと交差検証テストスコアの関係を示しています。
この可視化は通常、次の2つのことを示すために使用されます。

1. 推定量が増えることで推定量がどの程度恩恵を受けるか
（例：「十分なデータがある」か、オンラインで使用すると推定量が向上するかなど）。
1. 推定器が分散に起因する誤差とバイアスに起因する誤差のどちらに対してより敏感であるか。

以下の学習曲線を考えてみましょう
（Yellowbrickで生成されたものですが、scikit-learnドキュメントのPlotting Learning Curvesから生成されたものです）。

![](https://www.scikit-yb.org/en/latest/_images/learning_curve_sklearn_example.png)

より多くのデータが追加されるにつれて、学習スコアと交差検証スコアが一緒に収束する場合（左図に示す）、
モデルはおそらくより多くのデータの恩恵を受けないでしょう。
トレーニング・スコアが検証スコアよりもはるかに大きい場合、モデルはより効果的に一般化するために、
おそらくより多くのトレーニング例を必要とします。

曲線は平均スコアでプロットされていますが、クロス・バリデーション中の変動は、
すべてのクロス・バリデーションの平均の上と下の標準偏差を表す網掛けの領域で示されています。
モデルがバイアスのためにエラーを起こす場合、学習スコア曲線の周りには、おそらくより多くの変動があるでしょう。
モデルが分散のためにエラーを起こす場合、交差検証されたスコアの周囲には、より多くのばらつきがあるでしょう。

（注）学習曲線は、fit()メソッドとpredict()メソッド、および単一のスコアリングメトリックを持つすべての推定量について生成できます。
これには、以下の例で見るように、分類器、回帰器、およびクラスタリングが含まれます。

### Classification

次の例では、分類モデルの学習曲線を可視化する方法を示します。
DataFrameをロードし、カテゴリエンコーディングを実行した後、
各分割内のすべてのクラスが同じ比率で表現されることを保証するために、StratifiedKFoldクロスバリデーション戦略を作成します。
次に、分類器の精度とリコールの関係をよりよく理解するために、デフォルトのメトリックである精度ではなく、
f1_加重スコアリングメトリックを使用してビジュアライザーを適合させます。

In [ ]:
import numpy as np

from sklearn.model_selection import StratifiedKFold
from sklearn.naive_bayes import MultinomialNB
from sklearn.preprocessing import OneHotEncoder, LabelEncoder

from yellowbrick.datasets import load_game
from yellowbrick.model_selection import LearningCurve

# Load a classification dataset
X, y = load_game()

# Encode the categorical data
X = OneHotEncoder().fit_transform(X)
y = LabelEncoder().fit_transform(y)

# Create the learning curve visualizer
cv = StratifiedKFold(n_splits=12)
sizes = np.linspace(0.3, 1.0, 10)

# Instantiate the classification model and visualizer
model = MultinomialNB()
visualizer = LearningCurve(
    model, cv=cv, scoring='f1_weighted', train_sizes=sizes, n_jobs=4
)

visualizer.fit(X, y)        # Fit the data to the visualizer
visualizer.show()           # Finalize and render the figure

この学習曲線は、高いテスト変動性と約30,000インスタンスまでの低いスコアを示しますが、このレベルを超えると、
モデルは約0.6のF1スコアに収束し始めます。
訓練とテストのスコアがまだ収束していないことがわかりますので、
このモデルは潜在的にはより多くの訓練データの恩恵を受けるでしょう。

最後に、このモデルは主に分散（テスト・データのCVスコアはトレーニング・データよりも変動が大きい）による誤差に苦しんでいるので、モデルがオーバーフィットしている可能性があります。

### Regression

回帰の学習曲線を構築することは、直線的で非常に似ています。
下の例では、データをロードしてターゲットを選択した後、決定係数またはR2スコアに従って学習曲線スコアを探索します。

In [ ]:
from sklearn.linear_model import RidgeCV

from yellowbrick.datasets import load_energy
from yellowbrick.model_selection import LearningCurve

# Load a regression dataset
X, y = load_energy()

# Instantiate the regression model and visualizer
model = RidgeCV()
visualizer = LearningCurve(model, scoring='r2')

visualizer.fit(X, y)        # Fit the data to the visualizer
visualizer.show()           # Finalize and render the figure

この学習曲線は非常に高い変動性を示し、約350個のインスタンスまではかなり低いスコアを示しています。
非常に高いスコアで収束しているので、このモデルがより多くのデータの恩恵を受けられることは明らかです。
潜在的に、より多くのデータと正則化のためのより大きなアルファがあれば、
このモデルはテスト・データでの変動がはるかに少なくなるでしょう。

### Clustering

学習曲線はクラスタリングモデルにも機能し、
シルエットスコアや密度スコアのようなクラスタの形状や組織を指定するメトリクスを使用することができます。
メンバーシップが事前に分かっている場合は、
以下のようにランドスコアを使用してクラスタリング性能を比較することができます。

In [ ]:
from sklearn.cluster import KMeans
from sklearn.datasets import make_blobs

from yellowbrick.model_selection import LearningCurve

# Generate synthetic dataset with 5 random clusters
X, y = make_blobs(n_samples=1000, centers=5, random_state=42)

# Instantiate the clustering model and visualizer
model = KMeans()
visualizer = LearningCurve(model, scoring="adjusted_rand_score", random_state=42)

visualizer.fit(X, y)        # Fit the data to the visualizer
visualizer.show()           # Finalize and render the figure

残念ながら、ランダムデータでは、これらの曲線は非常に変動しますが、クラスタリングに特有の項目を指摘するのに役立ちます。
第一に、y軸が非常に狭いことに注意してください。
大まかに言えば、これらの曲線は収束しており、実際にはクラスタリングアルゴリズムが非常にうまく機能しています。
第二に、クラスタリングにおいて、データポイントの収束は必ずしも悪いことではありません；
実際、我々はより多くのデータが追加されても、トレーニングスコアとクロスバリデーションスコアが発散しないようにしたいと考えています。

### Quick Method

同じ機能は、関連するクイックメソッド `learning_curve` で実現できます。
このメソッドは、関連付けられた引数で `LearningCurve` オブジェクトを構築し、
それをフィットさせてから (オプションで) すぐに可視化を表示します。

In [ ]:
from sklearn.linear_model import RidgeCV

from yellowbrick.datasets import load_energy
from yellowbrick.model_selection import learning_curve

# Load a regression dataset
X, y = load_energy()

learning_curve(RidgeCV(), X, y, scoring='r2')

### UCIの他のデータセットから選んで学習曲線を見てみる

Concrete を使用して Regression で学習曲線を見る

In [ ]:
from sklearn.linear_model import RidgeCV

from yellowbrick.datasets import load_concrete
from yellowbrick.model_selection import LearningCurve

# Load a regression dataset
X, y = load_concrete()

# Instantiate the regression model and visualizer
model = RidgeCV()
visualizer = LearningCurve(model, scoring='r2')

visualizer.fit(X, y)        # Fit the data to the visualizer
visualizer.show()           # Finalize and render the figure


energy と同様にうまく学習がすすんでいる．

## 課題６

[Pythonで学ぶ統計的機械学習 - 密度推定](https://github.com/kanamori-takafumi/book_StatMachineLearn_with_Python/blob/master/ch14densityratio.ipynb) を試してみる

密度比の推定

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
from scipy.stats import norm
from scipy.spatial import distance
from sklearn.metrics.pairwise import rbf_kernel
import seaborn as sns
import pandas as pd

In [ ]:
# データ設定 
n, m = 100, 200
nu_mean, nu_sd = -0.5, 1
a_mean,  a_sd = 1, 0.8
newdat = np.linspace(-4,4,500).reshape(500,1)  # 予測点
tnu =  norm.pdf(newdat, nu_mean, nu_sd)            # 確率密度の計算
tde = (norm.pdf(newdat, a_mean, a_sd)+tnu)/2
tw = tnu/tde                                   # 予測点上での真の密度比

In [ ]:
# データ生成
nu = np.random.normal(loc=nu_mean, scale=nu_sd, size=n).reshape(n,1)
ma = np.random.binomial(m,0.5); mb = m-ma
de = np.r_[np.random.normal(loc=nu_mean,scale=nu_sd,size=ma).reshape(ma,1), np.random.normal(loc=a_mean, scale=a_sd, size=mb).reshape(mb,1)]

In [ ]:
class kernelDensityRatio:
    """
    kernel density-ratio estimator using Gaussian kernel
    gamma: bandwidth of Gaussian kernel
    lam: regularizaiton parameter
    """
    def __init__(self, gamma=None, lam=None):
        self.gamma = gamma             # カーネル幅
        self.lam = lam                 # 正則化パラメータ
        
    def fit(self, de, nu):             # 密度比推定
        if self.gamma is None:
            ma = nu.shape[0] + de.shape[0]
            idx = np.random.choice(ma,round(ma/2))
            self.gamma = (1/np.median(distance.pdist(np.r_[nu,de][idx,:])))**2
        if self.lam is None:
            self.lam = (min(nu.shape[0], de.shape[0]))**(-0.9)
        gamma = self.gamma; lam = self.lam
        n = de.shape[0]
        # グラム行列の計算
        Kdd = rbf_kernel(de, gamma=gamma)
        Kdn = rbf_kernel(de, nu, gamma=gamma)
        # 係数の推定
        Amat = Kdd + n*lam*np.identity(n)
        bvec = -np.mean(Kdn,1)/lam
        self.alpha = np.linalg.solve(Amat, bvec)
        self.de, self.nu = de, nu
        return self
    
    def predict(self, x):               # 予測点 x での密度比の値
        Wde =  np.dot(rbf_kernel(x, self.de, gamma=self.gamma), self.alpha)
        Wnu = np.mean(rbf_kernel(x, self.nu, gamma=self.gamma),1)/self.lam
        return np.maximum(Wde + Wnu,0)


In [ ]:
plt.plot(newdat,tw,lw=2)            # 真の密度比関数のプロット
# データ点のプロット
plt.scatter(nu.reshape(n,),np.repeat(0.3,n),marker='.',c='black',s=20)
plt.scatter(de.reshape(m,),np.repeat(0.1,m),marker='x',c='gray',s=20)
# 以下のカーネル幅で推定
gammas = np.array([0.01, 0.1, 1])
for g in gammas:
    dr = kernelDensityRatio(gamma=g)
    dr.fit(de,nu)                   # データへのフィッティング
    drp = dr.predict(newdat)        # 密度比の予測値
    plt.plot(newdat,drp, c='red',linestyle='dashed',lw=2)  # プロット
plt.show()

密度比推定のための交差検証法

In [ ]:
from scipy.spatial import distance        # distanceの計算
cvk = 5                                                      # 交差検証法のK
n, m = nu.shape[0], de.shape[0]          # データ数
# カーネル幅パラメータの候補を生成
idx = np.random.choice(n+m,round((n+m)/2))
gammas = 1/np.percentile(distance.pdist(np.r_[nu,de][idx,:]),[1,99])**2
gammas = np.logspace(np.log10(gammas.min()/100), np.log10(gammas.max()*100),10)
# 正則化パラメータ lambda の候補を生成 
lams = np.array([(min(n,m))**(-0.9)])
# モデルパラメータの候補
modelpars = np.array([(x,y) for x in gammas for y in lams])

In [ ]:
# それぞれのデータを5グループに分ける
inu = np.repeat(np.arange(cvk),np.ceil(n/cvk))
inu = inu[np.random.choice(n,n,replace=False)]
ide = np.repeat(np.arange(cvk),np.ceil(m/cvk))
ide = ide[np.random.choice(m,m,replace=False)]
cvloss = []
for gamma, lam in modelpars:
    tcvloss = []
    for k in np.arange(cvk):
        # トレーニングデータ
        trnu, trde = nu[inu!=k,:], de[ide!=k,:]
        # テストデータ
        tenu, tede = nu[inu==k,:], de[ide==k,:]
        # 指定されたモデルパラメータで密度比を推定
        kdr =kernelDensityRatio(gamma=gamma, lam=lam)
        kdr.fit(trde, trnu)                             # 推定
        wde = kdr.predict(tede)                         # de上で予測
        wnu = kdr.predict(tenu)                         # nu上で予測
        tcvloss.append(np.mean(wde**2)/2-np.mean(wnu))  # 二乗損失
    cvloss.append(np.mean(tcvloss))
# 最適なモデルパラメータ
optgamma,optlam = modelpars[np.argmin(cvloss),:]
optgamma,optlam

In [ ]:
plt.xscale('log'); plt.xlabel('gamma'); plt.ylabel('cv loss')
plt.scatter(gammas, cvloss)
plt.show()

In [ ]:
# 密度比の推定
kdr =kernelDensityRatio(gamma=optgamma, lam=optlam)
kdr.fit(de,nu)
kdr.predict(newdat)

共変量シフトの下での回帰分析

In [ ]:
# 真の回帰関数 
def f(x): 
    return (x+2)*(x-3)*x
# データ設定
ntr, mtr, sdtr   = 100, -1.4, 0.7
nte, mte, sdte = 100,  0.8, 0.8
# トレーニングデータ生成
xtr = np.random.normal(loc=mtr, scale=sdtr, size=ntr).reshape(ntr,1)
ytr = f(xtr) + np.random.normal(scale=2, size=ntr).reshape(ntr,1)
# テストデータ生成
xte = np.random.normal(loc=mte, scale=sdte, size=nte).reshape(nte,1)
yte = f(xte) + np.random.normal(scale=2, size=nte).reshape(nte,1)

In [ ]:
# トレーニングデータ点上での密度比を推定
kdr = kernelDensityRatio()
kdr.fit(xtr,xte)
pw = kdr.predict(xtr)
# 重み付き最小二乗法で回帰パラメータを推定
W = np.sqrt(np.diag(pw))
X =  sm.add_constant(xtr)
WX = np.dot(W,X); WY = np.dot(W, ytr)
estTheta = np.linalg.solve(np.dot(WX.T,WX), np.dot(WX.T,WY))
estTheta

二標本検定

In [ ]:
from sklearn.preprocessing import scale
from sklearn import datasets
d = datasets.load_breast_cancer()        # データ読込み
de = d.data[d.target==0]; de = scale(de) # スケーリング
nu = d.data[d.target==1]; nu = scale(nu) # スケーリング

In [ ]:
kdr =kernelDensityRatio() 
kdr.fit(de,nu)                               # 密度比の推定
L1distEst = np.mean(abs(1-kdr.predict(de)))  # L1距離の推定値
nperm = 10000                                # 並べ替え検定の繰り返し数
nde, nnu = de.shape[0], nu.shape[0]
dall = np.r_[de,nu]
permL1dist = []
for itr in np.arange(nperm):
    idx = np.random.choice(nde+nnu,nde,replace=False)   # データの並べ替え
    perm_de = dall[idx,:]
    perm_nu = np.delete(dall,idx,0)
    pdr =kernelDensityRatio()
    pdr.fit(perm_de, perm_nu)            # 並べ替えデータに対する密度比の推定
    permL1dist.append(np.mean(abs(1-pdr.predict(perm_de)))) # L1距離の推定
np.mean(L1distEst < np.array(permL1dist))          # 並べ替え検定による p値

`banana` データセットでの共変量シフト問題

In [ ]:
import os, glob, time, copy, random, zipfile

In [ ]:
os.listdir('./data/banana')

In [ ]:
# Train_dir, Test_dir
base_dir = './data/banana'
train_dir = base_dir + '/train'
test_dir  = base_dir + '/test'

# Check File Name
os.listdir(train_dir)[:5]

In [ ]:
# FilePath List
train_data_list = glob.glob(os.path.join(train_dir , 'banana_train_data_*.asc'))
train_label_list = glob.glob(os.path.join(train_dir , 'banana_train_labels_*.asc'))
test_data_list = glob.glob(os.path.join(test_dir, '*banana_test_data_*.asc'))
test_label_list = glob.glob(os.path.join(test_dir, '*banana_test_labels_*.asc'))

train_data_list = [string.replace('\\', '/') for string in train_data_list]
train_label_list = [string.replace('\\', '/') for string in train_label_list]
test_data_list = [string.replace('\\', '/') for string in test_data_list]
test_label_list = [string.replace('\\', '/') for string in test_label_list]

print(train_data_list)

In [ ]:
# 学習データ読み込み
f = open(train_data_list[0])
X_train = [s.lstrip() for s in f.read().split('\n')]
X_train = [ list(map(float, list(filter(lambda a: a != '', s.split(' '))))) for s in X_train ]
X_train = [ x for x in X_train if len(x) == 2 ]
print([i for i,x in enumerate(X_train) if len(x) != 2])
X_train = np.array(X_train, dtype=float)
print(X_train.shape)
print(X_train.dtype)
f.close()

f = open(train_label_list[0])
y_train = [ s.lstrip() for s in f.read().split('\n') ]
y_train = [ list(map(float, list(filter(lambda a: a != '', s.split(' '))))) for s in y_train ]
y_train = [ x for x in y_train if len(x) == 1 ]
print([i for i,x in enumerate(y_train) if len(x) != 1])
y_train = np.array(y_train, dtype=float)
print(y_train.shape)
print(y_train.dtype)
f.close()

In [ ]:
# テストデータ
f = open(test_data_list[0])
X_test = [s.lstrip() for s in f.read().split('\n')]
X_test = [ list(map(float, list(filter(lambda a: a != '', s.split(' '))))) for s in X_test ]
X_test = [ x for x in X_test if len(x) == 2 ]
print([i for i,x in enumerate(X_test) if len(x) != 2])
X_test = np.array(X_test, dtype=float)
print(X_test.shape)
print(X_test.dtype)
f.close()

f = open(test_label_list[0])
y_test = [ s.lstrip() for s in f.read().split('\n') ]
y_test = [ list(map(float, list(filter(lambda a: a != '', s.split(' '))))) for s in y_test ]
y_test = [ x for x in y_test if len(x) == 1 ]
print([i for i,x in enumerate(y_test) if len(x) != 1])
y_test = np.array(y_test, dtype=float)
print(y_test.shape)
print(y_test.dtype)
f.close()

In [ ]:
plt.scatter(X_test[:,0], X_test[:,1])
plt.scatter(X_train[:,0],X_train[:,1])
plt.show()

In [ ]:
# # トレーニングデータ点上での密度比を推定
# kdr = kernelDensityRatio()
# kdr.fit(X_train, y_train)
# pw = kdr.predict(X_train)
# # 重み付き最小二乗法で回帰パラメータを推定
# W = np.sqrt(np.diag(pw))
# X =  sm.add_constant(X_train)
# WX = np.dot(W,X); WY = np.dot(W, y_train)
# estTheta = np.linalg.solve(np.dot(WX.T,WX), np.dot(WX.T,WY))
# estTheta

In [ ]:
# 学習データのカーネル密度推定
x = X_train[:,0]; y = X_train[:,1]
df = pd.DataFrame(data=X_train, columns=['x1', 'x2'], dtype='float')
g = sns.jointplot(x="x1",y="x2",data=df, kind="kde")
g.plot_joint(plt.scatter, c="c", s=30, linewidth=1, marker="+")
g.ax_joint.collections[0].set_alpha(0)
g.set_axis_labels("$X_1$", "$X_2$")

In [ ]:
# テストデータのカーネル密度推定
df = pd.DataFrame(data=X_test, columns=['x1', 'x2'], dtype='float')
g = sns.jointplot(x="x1",y="x2",data=df, kind="kde")
g.plot_joint(plt.scatter, c="c", s=30, linewidth=1, marker="+")
g.ax_joint.collections[0].set_alpha(0)
g.set_axis_labels("$X_1$", "$X_2$")
